In [ ]:
from loguru import logger
from torch.utils.tensorboard import SummaryWriter
print("Device : " + device)
epochs = 15
learning_rate = 0.02 #way too high but hopefully quick
eval_steps = 3
log_dir = '..\\trained_models\\'
log_dir = Path(log_dir)
timestamp = datetime.now().strftime("%Y%m%d-%H%M")
log_dir = log_dir / timestamp
print(log_dir)
clean_dir(log_dir)

writer = SummaryWriter(log_dir=log_dir)
model = model.to(device)
optimizer_ = optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(epochs):
    train_loss = 0.0        
    model.train()    
    for batch in train_dataloader:
        optimizer_.zero_grad()
        input, target = batch
        input = input.to(device)
        target = target.to(device)
        output = model(input)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer_.step()
        train_loss += loss.data.item()
    train_loss /= len(train_dataloader.dataset)
    print(f"Epoch : {epoch} - train loss ={train_loss}")
    writer.add_scalar("Loss/train", train_loss, epoch)

    model.eval()
    test_loss = 0.0
    test_accuracy = 0.0
    for _ in range(eval_steps):
        input, target = next(iter(test_dataloader))
        input = input.to(device)        
        target = target.to(device)
        output = model(input)
        loss = loss_fn(output, target)
        test_loss += loss.data.item()
        test_accuracy += (output.argmax(dim=1) == target).sum()
    datasize = eval_steps * test_dataloader.batch_size
    test_loss /= datasize
    test_accuracy /= datasize
    writer.add_scalar("Loss/test", test_loss, epoch)
    writer.add_scalar("Loss/accuracy", test_accuracy, epoch)    
    print(f"testloss :{test_loss} -  test accuracy :{test_accuracy}")
write_gin(log_dir)




In [ ]:

import torch
from torch import nn


# Define model
class oCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

model = CNN().to(device)
model.cuda()
print(model)